In [1]:
def text2paragraphs(filename, min_size=1):
    """ A text contained in the file 'filename' will be read 
    and chopped into paragraphs.
    Paragraphs with a string length less than min_size will be ignored.
    A list of paragraph strings will be returned"""
    
    txt = open(filename).read()
    paragraphs = [para for para in txt.split("\n\n") if len(para) > min_size]
    return paragraphs


In [2]:
# the position of lables is very important
# it corresponds to a novel by that author within "files"
# the position of the author is also relevant, as it will correspond to metrics
# i.e. Samuel Butler's metrics are always returned in position 1
labels = ['Virginia Woolf', 'Samuel Butler', 'Herman Melville', 
          'David Herbert Lawrence', 'Daniel Defoe', 'James Joyce']


# names of books we have to train our machine model
files = ['night_and_day_virginia_woolf.txt', 'the_way_of_all_flash_butler.txt',
         'moby_dick_melville.txt', 'sons_and_lovers_lawrence.txt',
         'robinson_crusoe_defoe.txt', 'james_joyce_ulysses.txt']

# location of our books
path = "books/"
data = []
targets = []
counter = 0

# loop across all files we have downloaded
for fname in files:
    paras = text2paragraphs(path + fname, min_size=150) # return a book with paragraphs over 150 chars in a list
    data.extend(paras)
    targets += [counter] * len(paras)
    counter += 1


In [3]:
# cell is useless, because train_test_split will do the shuffling!

import random

data_targets = list(zip(data, targets))
# create random permutation on list:
data_targets = random.sample(data_targets, len(data_targets))

data, targets = list(zip(*data_targets))


In [4]:
from sklearn.model_selection import train_test_split

res = train_test_split(data, targets, 
                       train_size=0.8,
                       test_size=0.2,
                       random_state=42)
train_data, test_data, train_targets, test_targets = res 


In [5]:
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS

from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

vectorizer = CountVectorizer(stop_words=ENGLISH_STOP_WORDS)

vectors = vectorizer.fit_transform(train_data)

# creating a classifier
classifier = MultinomialNB(alpha=.01)
classifier.fit(vectors, train_targets)

vectors_test = vectorizer.transform(test_data)

predictions = classifier.predict(vectors_test)
accuracy_score = metrics.accuracy_score(test_targets, 
                                        predictions)
f1_score = metrics.f1_score(test_targets, 
                            predictions, 
                            average='macro')

print("accuracy score: ", accuracy_score)
print("F1-score: ", f1_score)


accuracy score:  0.9134458356015243
F1-score:  0.9111786431106483


In [6]:
# we want to use paragraphs from this 2nd Virginia Wolf 
paras = text2paragraphs(path + "the_voyage_out_virginia_woolf.txt", min_size=250)

# start on paragraph 100 and go to paragraph 500
first_para, last_para = 100, 500
vectors_test = vectorizer.transform(paras[first_para: last_para]) # pass a list of strings that will be used to make predictions against
#vectors_test = vectorizer.transform(["To be or not to be"])

predictions = classifier.predict(vectors_test) # make our predictions
print(predictions)
targets = [0] * (last_para - first_para)
accuracy_score = metrics.accuracy_score(targets, 
                                        predictions)
precision_score = metrics.precision_score(targets, 
                                          predictions, 
                                          average='macro')

f1_score = metrics.f1_score(targets, 
                            predictions, 
                            average='macro')

print("accuracy score: ", accuracy_score)
print("precision score: ", accuracy_score)
print("F1-score: ", f1_score)


[5 0 5 5 0 5 5 0 2 5 0 0 0 0 0 0 0 0 1 0 1 0 0 5 0 5 0 0 0 0 0 0 5 5 2 5 0
 2 5 5 0 0 3 0 0 5 2 0 0 0 2 0 2 5 2 2 0 0 0 0 0 5 5 0 0 2 0 0 0 0 5 2 1 5
 0 0 1 0 0 2 2 3 0 2 2 0 5 3 0 5 0 0 0 0 0 5 1 0 1 0 5 3 5 5 2 5 5 5 2 0 5
 0 0 0 0 0 0 1 2 0 0 0 2 0 1 2 2 0 5 5 0 0 0 0 5 0 0 5 1 3 0 0 0 0 0 2 0 0
 0 0 5 0 5 5 5 5 0 1 1 0 0 0 0 0 0 5 0 1 0 0 0 0 5 5 5 0 2 0 5 0 0 0 0 0 0
 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 5 1 0 0 3 1 5 5 5 0 5 0 5 0 0 0 0 5 0
 0 5 5 0 0 0 0 0 3 0 0 0 0 2 1 0 5 5 5 1 0 1 5 0 0 0 5 0 1 0 1 0 0 0 0 1 3
 1 0 0 5 0 5 5 2 0 0 0 0 5 0 2 2 0 1 0 0 0 0 0 0 3 0 4 0 0 0 0 1 0 0 0 0 1
 1 0 0 5 5 0 5 0 0 0 3 0 5 0 0 0 0 5 3 1 3 0 0 3 0 0 0 0 0 0 3 0 5 5 0 0 5
 5 3 5 0 3 0 0 0 1 0 1 0 0 0 0 2 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 2 2 2 3 0 3
 0 1 0 0 0 0 2 0 0 0 0 0 0 0 3 0 0 0 0 0 1 5 0 0 0 0 0 0 0 0]
accuracy score:  0.605
precision score:  0.605
F1-score:  0.1256490134994808


In [7]:
# perform a probability test
predictions = classifier.predict_proba(vectors_test)
print(predictions)


[[7.10153836e-003 4.29924364e-002 3.30133198e-007 2.58344433e-005
  1.24413440e-013 9.49879861e-001]
 [9.87314210e-001 2.16199122e-004 9.68431117e-003 6.17872042e-011
  6.51173943e-015 2.78527944e-003]
 [4.70001468e-001 1.42367318e-009 1.75188882e-012 1.20814861e-008
  3.06937118e-014 5.29998519e-001]
 ...
 [9.99999999e-001 7.91022575e-010 4.90934691e-041 3.89383156e-022
  6.84008597e-040 1.48440660e-035]
 [9.99999987e-001 5.58598842e-014 5.69277142e-020 3.08362859e-035
  9.79834992e-055 1.33270655e-008]
 [1.00000000e+000 1.62503634e-049 1.63470591e-068 1.39137742e-086
  1.20658613e-114 1.61081544e-054]]


In [8]:
for i in range(0, 10):
    print(predictions[i], paras[i+first_para])


[7.10153836e-03 4.29924364e-02 3.30133198e-07 2.58344433e-05
 1.24413440e-13 9.49879861e-01] "That's the painful thing about pets," said Mr. Dalloway; "they die. The
first sorrow I can remember was for the death of a dormouse. I regret to
say that I sat upon it. Still, that didn't make one any the less sorry.
Here lies the duck that Samuel Johnson sat on, eh? I was big for my
age."
[9.87314210e-01 2.16199122e-04 9.68431117e-03 6.17872042e-11
 6.51173943e-15 2.78527944e-03] "Please tell me--everything." That was what she wanted to say. He had
drawn apart one little chink and showed astonishing treasures. It seemed
to her incredible that a man like that should be willing to talk to her.
He had sisters and pets, and once lived in the country. She stirred her
tea round and round; the bubbles which swam and clustered in the cup
seemed to her like the union of their minds.
[4.70001468e-01 1.42367318e-09 1.75188882e-12 1.20814861e-08
 3.06937118e-14 5.29998519e-01] The talk meanwhile raced pa